<a href="https://colab.research.google.com/github/ikramebakkari/C-Programming-Project/blob/main/CudaTD1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0x4ghx1c
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0x4ghx1c
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=8def43b495941275d2626e2a8be97ba93ca3726d7bd1d1daa324bee35f0ea536
  Stored in directory: /tmp/pip-ephem-wheel-cache-5j6fmr3w/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [2]:
!nvcc  --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [10]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [12]:
%%cu
#include <stdio.h>
#include <stdlib.h>
    // Execice 1 : Addition de deux entiers
__global__ void add (int *a, int *b, int *c) {
*c = *a + *b;
}
int main() {
    int a, b, c;
    // host copies of variables a, b & c
    int *d_a, *d_b, *d_c;
    int size = sizeof(int);
    // Allocate space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);
    // Setup input values
    c = 0;
    a = 3;
    b = 5;

    // Copy inputs to device
    cudaMemcpy (d_a, &a, size, cudaMemcpyHostToDevice);
    cudaMemcpy (d_b, &b, size, cudaMemcpyHostToDevice);

    // Launch add() kernel on GPU
    add<<<1,1>>>(d_a, d_b, d_c);
    // Copy result back to host
    cudaError err = cudaMemcpy (&c, d_c, size, cudaMemcpyDeviceToHost);

    if(err!=cudaSuccess) {
        printf("CUDA error copying to Host: %s\n", cudaGetErrorString(err));
    }

    printf("first value is %d\n", a);
    printf("secon value is %d\n", b);
    printf("result is %d\n", c);
    printf("error here %s\n", cudaGetErrorString(err));
    // Cleanup
    cudaFree (d_a); cudaFree (d_b); cudaFree (d_c) ;
    return a+b;
}

first value is 3
secon value is 5
result is 8
error here no error



In [21]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 10 // Ou toute autre valeur que vous souhaitez pour la taille des vecteurs
__global__ void add(int *a, int *b, int *c) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int a[N], b[N], c[N];
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocation de mémoire pour les vecteurs sur le GPU
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Initialisation des vecteurs a et b sur le CPU
    for (int i = 0; i < N; ++i) {
        a[i] = i;
        b[i] = i * 2;
    }

    // Copie des vecteurs a et b du CPU au GPU
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    int numBlocks = 1;
    int threadsPerBlock = N;
    add<<<numBlocks, threadsPerBlock>>>(d_a, d_b, d_c);

    // Copie du résultat du GPU au CPU
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Affichage du résultat
    printf("Vecteur résultant de l'addition : \n");
    for (int i = 0; i < N; ++i) {
        printf("%d + %d = %d\n", a[i], b[i], c[i]);
    }

    // Libération de la mémoire allouée sur le GPU
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}


Vecteur résultant de l'addition : 
0 + 0 = 0
1 + 2 = 3
2 + 4 = 6
3 + 6 = 9
4 + 8 = 12
5 + 10 = 15
6 + 12 = 18
7 + 14 = 21
8 + 16 = 24
9 + 18 = 27



In [23]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 4 // Taille des matrices (N x N)

__global__ void matrixMultiply(int *a, int *b, int *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int sum = 0;
    for (int k = 0; k < n; k++) {
        sum += a[row * n + k] * b[k * n + col];
    }

    c[row * n + col] = sum;
}

int main(void) {
    int a[N][N], b[N][N], c[N][N];
    int *dev_a, *dev_b, *dev_c;

    // Allocation de mémoire sur le GPU
    cudaMalloc((void **)&dev_a, N * N * sizeof(int));
    cudaMalloc((void **)&dev_b, N * N * sizeof(int));
    cudaMalloc((void **)&dev_c, N * N * sizeof(int));

    // Initialisation des matrices a et b sur le CPU
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            a[i][j] = i * N + j;
            b[i][j] = j * N + i;
        }
    }

    // Copie des matrices du CPU vers le GPU
    cudaMemcpy(dev_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Définir la taille des blocs et des grilles
    dim3 threadsPerBlock(N, N);
    dim3 numBlocks(1, 1);

    // Appel du kernel pour la multiplication des matrices
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(dev_a, dev_b, dev_c, N);

    // Copie du résultat du GPU vers le CPU
    cudaMemcpy(c, dev_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Affichage du résultat
    printf("Résultat de la multiplication :\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d ", c[i][j]);
        }
        printf("\n");
    }

    // Libération de la mémoire allouée sur le GPU
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


Résultat de la multiplication :
14 38 62 86 
38 126 214 302 
62 214 366 518 
86 302 518 734 



In [25]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#define N 5 // Taille des matrices (N x N)
#define blockSize 2 // Taille du bloc de threads

__global__ void matrixMultiply(int *a, int *b, int *c, int n) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int sum = 0;
    for (int k = 0; k < n; k++) {
        int a_element = a[row * n + k];
        int b_element = b[k * n + col];
        sum += a_element * b_element;
    }

    if (row < n && col < n) {
        c[row * n + col] = sum;
    }
}

int main(void) {
    int a[N][N], b[N][N], c[N][N];
    int *dev_a, *dev_b, *dev_c;

    // Allocation de mémoire sur le GPU
    cudaMalloc((void **)&dev_a, N * N * sizeof(int));
    cudaMalloc((void **)&dev_b, N * N * sizeof(int));
    cudaMalloc((void **)&dev_c, N * N * sizeof(int));

    // Initialisation des matrices a et b sur le CPU
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            a[i][j] = i * N + j;
            b[i][j] = j * N + i;
        }
    }

    // Copie des matrices du CPU vers le GPU
    cudaMemcpy(dev_a, a, N * N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, N * N * sizeof(int), cudaMemcpyHostToDevice);

    // Définir la taille des blocs et des grilles
    dim3 threadsPerBlock(blockSize, blockSize);
    dim3 numBlocks((N + blockSize - 1) / blockSize, (N + blockSize - 1) / blockSize);

    // Appel du kernel pour la multiplication des matrices
    matrixMultiply<<<numBlocks, threadsPerBlock>>>(dev_a, dev_b, dev_c, N);

    // Copie du résultat du GPU vers le CPU
    cudaMemcpy(c, dev_c, N * N * sizeof(int), cudaMemcpyDeviceToHost);

    // Affichage du résultat
    printf("Résultat de la multiplication :\n");
    for (int i = 0; i < N; ++i) {
        for (int j = 0; j < N; ++j) {
            printf("%d ", c[i][j]);
        }
        printf("\n");
    }

    // Libération de la mémoire allouée sur le GPU
    cudaFree(dev_a);
    cudaFree(dev_b);
    cudaFree(dev_c);

    return 0;
}


Résultat de la multiplication :
30 80 130 180 230 
80 255 430 605 780 
130 430 730 1030 1330 
180 605 1030 1455 1880 
230 780 1330 1880 2430 



In [26]:
%%cu

#include <stdio.h>

#define N 10 // Taille d'une dimension de la matrice

// Kernel pour la multiplication de matrices
__global__ void matrixMultiply(int *a, int *b, int *c) {
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    int sum = 0;
    for (int k = 0; k < N; k++) {
        sum += a[row * N + k] * b[k * N + col];
    }

    c[row * N + col] = sum;
}

int main(void) {
    int a[N][N], b[N][N], c[N][N];
    int *gpu_a, *gpu_b, *gpu_c;
    int size = N * N * sizeof(int);

    // Allocation de l espace pour les copies sur le GPU de a, b et c
    cudaMalloc((void **)&gpu_a, size);
    cudaMalloc((void **)&gpu_b, size);
    cudaMalloc((void **)&gpu_c, size);

    // Initialisation des valeurs dans les matrices a et b
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            a[i][j] = j * i + i;
            b[i][j] = j * i + i;
        }
    }

    // Copie des matrices d'entrée vers le GPU
    cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

    dim3 blockDim(2, 2); // Taille du bloc de threads
    dim3 gridDim((N + blockDim.x - 1) / blockDim.x, (N + blockDim.y - 1) / blockDim.y); // Taille de la grille

    // Appel du kernel
    matrixMultiply<<<gridDim, blockDim>>>(gpu_a, gpu_b, gpu_c);

    // Copie du résultat de retour vers le CPU
    cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);

    // Affichage du résultat
    printf("Résultat de la multiplication :\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", c[i][j]);
        }
        printf("\n");
    }

    // Libération de la mémoire allouée sur le GPU
    cudaFree(gpu_a);
    cudaFree(gpu_b);
    cudaFree(gpu_c);

    return 0;
}

Résultat de la multiplication :
0 0 0 0 0 0 0 0 0 0 
330 660 990 1320 1650 1980 2310 2640 2970 3300 
660 1320 1980 2640 3300 3960 4620 5280 5940 6600 
990 1980 2970 3960 4950 5940 6930 7920 8910 9900 
1320 2640 3960 5280 6600 7920 9240 10560 11880 13200 
1650 3300 4950 6600 8250 9900 11550 13200 14850 16500 
1980 3960 5940 7920 9900 11880 13860 15840 17820 19800 
2310 4620 6930 9240 11550 13860 16170 18480 20790 23100 
2640 5280 7920 10560 13200 15840 18480 21120 23760 26400 
2970 5940 8910 11880 14850 17820 20790 23760 26730 29700 



In [36]:
%%cu

#include <stdio.h>

#define N 10 // Taille d'une dimension de la matrice
#define TILE_SIZE 2 // Taille de la tuile

// Kernel pour la multiplication de matrices avec la méthode de tuile
__global__ void matrixMultiplyTiled(int *a, int *b, int *c) {
    __shared__ int tile_a[TILE_SIZE][TILE_SIZE];
    __shared__ int tile_b[TILE_SIZE][TILE_SIZE];

    int row = blockIdx.y * TILE_SIZE + threadIdx.y;
    int col = blockIdx.x * TILE_SIZE + threadIdx.x;

    int sum = 0;
    for (int t = 0; t < N / TILE_SIZE; t++) {
        tile_a[threadIdx.y][threadIdx.x] = a[row * N + t * TILE_SIZE + threadIdx.x];
        tile_b[threadIdx.y][threadIdx.x] = b[(t * TILE_SIZE + threadIdx.y) * N + col];
        __syncthreads();

        for (int k = 0; k < TILE_SIZE; k++) {
            sum += tile_a[threadIdx.y][k] * tile_b[k][threadIdx.x];
        }
        __syncthreads();
    }

    c[row * N + col] = sum;
}

int main(void) {
    int a[N][N], b[N][N], c[N][N];
    int *gpu_a, *gpu_b, *gpu_c;
    int size = N * N * sizeof(int);

    // Allocation de l'espace pour les copies sur le GPU de a, b et c
    cudaMalloc((void **)&gpu_a, size);
    cudaMalloc((void **)&gpu_b, size);
    cudaMalloc((void **)&gpu_c, size);
    cudaMemcpy(gpu_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(gpu_b, b, size, cudaMemcpyHostToDevice);

    dim3 blockDim(TILE_SIZE, TILE_SIZE); // Taille de la tuile
    dim3 gridDim(N / TILE_SIZE, N / TILE_SIZE); // Taille de la grille

    // Appel du kernel
    matrixMultiplyTiled<<<gridDim, blockDim>>>(gpu_a, gpu_b, gpu_c);

    // Copie du résultat de retour vers le CPU
    cudaMemcpy(c, gpu_c, size, cudaMemcpyDeviceToHost);
    // Libération de la mémoire allouée sur le GPU
    cudaFree(gpu_a);
    cudaFree(gpu_b);
    cudaFree(gpu_c);
        // Affichage du résultat
    printf("Résultat de la multiplication en tiles :\n");
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            printf("%d ", c[i][j]);
        }
        for (int j = 0; j < N; j++) {
            printf("ici a  %d ", a[i][j]);
        }
        for (int j = 0; j < N; j++) {
            printf("ici b  %d ", b[i][j]);
        }
        printf("\n");
    }

    return 0;
}

Résultat de la multiplication en tiles :
-1065569792 0 1610297120 1338185920 -1440385248 1338185920 0 0 -1016041280 0 ici a  0 ici a  0 ici a  0 ici a  0 ici a  0 ici a  0 ici a  3167716 ici a  0 ici a  -1016041280 ici a  32767 ici b  -1415766019 ici b  23472 ici b  1630256366 ici b  30749 ici b  -1415110624 ici b  23472 ici b  -1415131104 ici b  23472 ici b  0 ici b  0 
-1355373206 -1002870621 -1453010116 747406027 -556822904 602613916 -2068742418 -1460436735 1728850658 -1597799398 ici a  1627922298 ici a  30749 ici a  1630160240 ici a  30749 ici a  -1016041440 ici a  32767 ici a  -1016041424 ici a  32767 ici a  1630216721 ici a  30749 ici b  -1016040136 ici b  32767 ici b  1630409440 ici b  30749 ici b  1630015344 ici b  30749 ici b  1 ici b  0 ici b  1630410752 ici b  30749 
-1201366574 -990483680 -1135452132 -712404102 324630585 17068803 257816118 -349868351 1 0 ici a  2 ici a  0 ici a  1630161984 ici a  30749 ici a  1 ici a  0 ici a  0 ici a  0 ici a  1 ici a  0 ici b  1 ici b  32